# Modeling fixed rate bonds {#fixedratebonds}


In this chapter we will consider a simple example to model fixed rate bonds. Let's consider a hypothetical bond with a par value of 100, that pays 6% coupon semi-annually issued on January 15th, 2015 and set to mature on January 15th, 2016. The bond will pay a coupon on July 15th, 2015 and January 15th, 2016. The par amount of 100 will also be paid on the January 15th, 2016.

To make things simpler, lets assume that we know the spot rates of the treasury as of January 15th, 2015. The annualized spot rates are 0.5% for 6 months and 0.7% for 1 year point. Lets calculate the net present value of the cash flows directly as shown below.


In [1]:
3/pow(1+0.005, 0.5) + (100 + 3)/(1+0.007)

105.27653992490681

Here, we discounted the coupon and par payments with the appropriate spot rates. Now we will replicate this calculation using the QuantLib framework. Let's start by importing the `QuantLib` module.

In [2]:
from QuantLib import *

As a first step, we need to construct a yield curve with the given spot rates. This is done using the `ZeroCurve` class as discussed in an [earlier chapter](#yc-construction). 

In [3]:
calc_date = Date(15, 1, 2015)
Settings.instance().evaluationDate = calc_date
spot_dates = [Date(15, 1, 2015), Date(15, 7, 2015), Date(15, 1, 2016)]
spot_rates = [0.0, 0.005, 0.007]
day_count = Thirty360()
calendar = UnitedStates()
interpolation = Linear()
compounding = Compounded
compounding_frequency = Annual
spot_curve = ZeroCurve(spot_dates, spot_rates, day_count, calendar, 
                       interpolation, compounding, compounding_frequency)
spot_curve_handle = YieldTermStructureHandle(spot_curve)

As a next step, we are going to build a fixed rate bond object. In order to construct the `FixedRateBond` object, we will create a schedule for the coupon payments. Creation of `Schedule` object was discussed in an [earlier chapter](#quantlib-basics). 

In [4]:
issue_date = Date(15, 1, 2015)
maturity_date = Date(15, 1, 2016)
tenor = Period(Semiannual)
calendar = UnitedStates()
business_convention = Unadjusted
date_generation = DateGeneration.Backward
month_end = False
schedule = Schedule (issue_date, maturity_date, tenor, 
                     calendar, business_convention, 
                     business_convention , date_generation, 
                     month_end)

Let us print the schedule to check if it is in agreement with what we expect it to be. 

In [5]:
list(schedule)

[Date(15,1,2015), Date(15,7,2015), Date(15,1,2016)]

Now that we have the schedule, we can create the `FixedRateBond` object. The `FixedRateBond` constructor has the following signature.

    FixedRateBond(Natural settlementDays,
                  Real faceAmount,
                  const Schedule& schedule,
                  const std::vector<Rate>& coupons,
                  const DayCounter& accrualDayCounter,
                  BusinessDayConvention paymentConvention = Following,
                  Real redemption = 100.0,
                  const Date& issueDate = Date(),
                  const Calendar& paymentCalendar = Calendar(),
                  const Period& exCouponPeriod = Period(),
                  const Calendar& exCouponCalendar = Calendar(),
                  const BusinessDayConvention exCouponConvention = Unadjusted,
                  bool exCouponEndOfMonth = false)

Let us create the `FixedRateBond` object below.

In [6]:
coupon_rate = .06
coupons = [coupon_rate]
settlement_days = 0
face_value = 100

fixed_rate_bond = FixedRateBond(settlement_days, 
                                face_value, 
                                schedule,
                                coupons, 
                                day_count)

Now the we have the fixed rate bond instrument, we need a valuation engine in order to price this bond. The fixed rate bond can be priced using a `DiscountingBondEngine`. The `DiscountingBondEngine` takes the yield curve object as an argument in its constructor. The `setPricingEngine` method in the fixed rate bond instrument is used to set the pricing engine.

In [7]:
bond_engine = DiscountingBondEngine(spot_curve_handle)
fixed_rate_bond.setPricingEngine(bond_engine)

Now, the net present value of the bond can be extracted using the `NPV` method.

In [8]:
fixed_rate_bond.NPV()

105.27653992490683

In [9]:
fixed_rate_bond.cleanPrice()

105.27653992490683

We can also obtain various other analytics for the bond.

In [10]:
fixed_rate_bond.accruedAmount()

0.0

In [11]:
fixed_rate_bond.dirtyPrice()

105.27653992490683

In [12]:
fixed_rate_bond.bondYield(day_count,
                          compounding,
                          compounding_frequency)

0.006971154634952549